In [8]:
import pandas as pd
import numpy as np

In [9]:
data = pd.read_csv('phev.csv')

In [10]:
data['Car price (US $)'] = (data['Car price (US $)'] * 0.93).round(2)
data.rename(columns={'Car price (US $)': 'Car price (EUR)'}, inplace=True)

# Преобразование Battery-only range из миль в километры
data['Battery-only range (miles)'] = (data['Battery-only range (miles)'] * 1.60934).round(1)
data.rename(columns={'Battery-only range (miles)': 'Battery-only range (km)'}, inplace=True)

# Преобразование kWh/100 miles в kWh/100 km
data['kWh / 100 miles (if operating on battery only)'] = (data['kWh / 100 miles (if operating on battery only)'] / 1.60934).round(1)
data.rename(columns={'kWh / 100 miles (if operating on battery only)': 'kWh / 100 km (battery only)'}, inplace=True)

# Создаем значения с нормальным распределением (среднее = 2.6, std = 0.3)
frontal_area = np.random.normal(loc=2.6, scale=0.3, size=len(data))

# Обрезаем значения до диапазона [2.0, 3.2] и округляем
frontal_area = np.clip(frontal_area, 2.0, 3.2).round(1)

# Добавляем столбец в DataFrame
data["frontal_area_m2"] = frontal_area

data[['firm_name', 'model_name']] = data['Model'].str.split(' ', n=1, expand=True)


# 3) Переставляем столбцы в желаемом порядке
cols = [
    'firm_name',
    'model_name',
    'Battery-only range (km)',
    'MPG (if operating on gas only)',
    'Car price (EUR)',
    'kWh / 100 km (battery only)',
    'frontal_area_m2',
    'Weight (kg)'
]
def calculate_car_weight(row):
    """
    Генерирует правдоподобную массу автомобиля на основе его характеристик.
    Алгоритм учитывает тип автомобиля, технические характеристики и рыночный класс.
    """
    # Базовый вес по типу автомобиля
    vehicle_type = 'other'
    model_lower = row['model_name'].lower()
    
    if 'x5' in model_lower or 'gle' in model_lower:
        vehicle_type = 'suv'
        base_weight = 2100
    elif 'prius' in model_lower or 'leaf' in model_lower:
        vehicle_type = 'compact'
        base_weight = 1500
    elif 's60' in model_lower or '3 series' in model_lower:
        vehicle_type = 'sedan'
        base_weight = 1700
    elif 'model s' in model_lower or 'e-tron' in model_lower:
        vehicle_type = 'premium'
        base_weight = 2200
    else:
        # Определение по frontal area
        base_weight = 1600 + (row['frontal_area_m2'] - 2.3) * 300

    # Корректировка веса на основе характеристик
    battery_weight = row['Battery-only range (km)'] * 0.8  # 0.8 кг/км запаса хода
    efficiency_penalty = (100 - row['MPG (if operating on gas only)']) * 2
    luxury_factor = np.log(row['Car price (EUR)'] / 30000) * 100
    frontal_area_weight = row['frontal_area_m2'] * 150
    
    # Формула расчета
    calculated_weight = (
        base_weight 
        + battery_weight 
        + frontal_area_weight 
        + luxury_factor 
        - efficiency_penalty
    )
    
    # Добавление случайного отклонения
    noise = np.random.normal(0, 50)
    
    # Ограничение диапазона
    final_weight = np.clip(calculated_weight + noise, 1000, 3000)
    
    return int(final_weight)

# Применение функции ко всему датасету
def generate_weights_for_dataset(df):   
    df['Weight (kg)'] = df.apply(calculate_car_weight, axis=1)
    return df

data = generate_weights_for_dataset(data)

data = data[cols]
# Сохранение результата
data.to_csv('phev_converted.csv', index=False)

# Просмотр первых строк
data.head()


,firm_name,model_name,Battery-only range (km),MPG (if operating on gas only),Car price (EUR),kWh / 100 km (battery only),frontal_area_m2,Weight (kg)
0,BMW,X5 xDrive45e,86.9,50.0,60450.0,27.3,2.6,2454
1,Mercedes-Benz,GLE 350de,106.2,60.0,69750.0,23.6,2.2,2557
2,Toyota,Prius Plug-In Hybrid,70.8,52.0,32550.0,16.2,2.4,1771
3,Volvo,S60 Recharge,66.0,31.0,47662.5,27.3,2.6,2155
4,Volvo,V60 Recharge,66.0,31.0,65611.5,27.3,2.4,2022
